### Vehicle Detection Pipeline

In [2]:
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
import cv2
import glob
import time
from sklearn.svm import LinearSVC
from sklearn.preprocessing import StandardScaler
from skimage.feature import hog
from scipy.ndimage.measurements import label
%matplotlib inline

# NOTE: the next import is only valid for scikit-learn version <= 0.17
# for scikit-learn >= 0.18 use:
from sklearn.model_selection import train_test_split
# from sklearn.cross_validation import train_test_split


# Define a function to return HOG features and visualization
def get_hog_features(img, orient, pix_per_cell, cell_per_block, 
                        vis=False, feature_vec=True):
    # Call with two outputs if vis==True
    if vis == True:
        features, hog_image = hog(img, orientations=orient, 
                                  pixels_per_cell=(pix_per_cell, pix_per_cell),
                                  cells_per_block=(cell_per_block, cell_per_block), 
                                  transform_sqrt=True, 
                                  visualise=vis, feature_vector=feature_vec)
        return features, hog_image
    # Otherwise call with one output
    else:      
        features = hog(img, orientations=orient, 
                       pixels_per_cell=(pix_per_cell, pix_per_cell),
                       cells_per_block=(cell_per_block, cell_per_block), 
                       transform_sqrt=False, 
                       visualise=vis, feature_vector=feature_vec)
#         print(np.any(np.isnan(img)))
        return features

# Define a function to compute binned color features  
def bin_spatial(img, size=(32, 32)):
    # Use cv2.resize().ravel() to create the feature vector
    features = cv2.resize(img, size).ravel() 
    # Return the feature vector
    return features

# Define a function to compute color histogram features 
# NEED TO CHANGE bins_range if reading .png files with mpimg!
def color_hist(img, nbins=32, bins_range=(0, 256)):
    # Compute the histogram of the color channels separately
    channel1_hist = np.histogram(img[:,:,0], bins=nbins, range=bins_range)
    channel2_hist = np.histogram(img[:,:,1], bins=nbins, range=bins_range)
    channel3_hist = np.histogram(img[:,:,2], bins=nbins, range=bins_range)
    # Concatenate the histograms into a single feature vector
    hist_features = np.concatenate((channel1_hist[0], channel2_hist[0], channel3_hist[0]))
    # Return the individual histograms, bin_centers and feature vector
    return hist_features

# Define a function to extract features from a list of images
# Have this function call bin_spatial() and color_hist()
def extract_features(imgs, color_space='RGB', spatial_size=(32, 32),
                        hist_bins=32, orient=9, 
                        pix_per_cell=8, cell_per_block=2, hog_channel=0,
                        spatial_feat=True, hist_feat=True, hog_feat=True):
    # Create a list to append feature vectors to
    features = []
    # Iterate through the list of images
    for file in imgs:
        file_features = []
        # Read in each one by one
        image = mpimg.imread(file)
        feature_image = np.zeros_like(image)
        feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2YCrCb)
#         feature_image = image
#         r_rgb = image[:,:,0]
#         s_hsv = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)[:,:,1]
#         u_luv = cv2.cvtColor(image, cv2.COLOR_RGB2LUV)[:,:,1]
#         feature_image[:,:,0] = r_rgb
#         feature_image[:,:,1] = s_hsv
#         feature_image[:,:,2] = u_luv
#         plt.figure()
#         plt.imshow(feature_image)
        # use original image in spacial features
        if spatial_feat == True:
            spatial_features = bin_spatial(feature_image, size=spatial_size)
            
#             file_features.append(spatial_features)
        # use multiple color spaces for hist
        if hist_feat == True:
            # Apply color_hist()
            hist_features = color_hist(feature_image, nbins=hist_bins)
            
#             file_features.append(hist_features)
        # use multiple color spaces for hog
        if hog_feat == True:
        # Call get_hog_features() with vis=False, feature_vec=True
#             print('feature Image', feature_image)
            hog_features = []
            for channel in range(feature_image.shape[2]):
                hog_features.append(get_hog_features(feature_image[:,:,channel], 
                                    orient, pix_per_cell, cell_per_block, 
                                    vis=False, feature_vec=True))
                
#             hog_features.append(get_hog_features(feature_image[:,:,0], 
#                     orient, pix_per_cell, cell_per_block, 
#                     vis=False, feature_vec=True))    
                
            hog_features = np.ravel(hog_features)        
            
            # Append the new feature vector to the features list
#             file_features.append(hog_features)

        features.append(np.concatenate((spatial_features, hist_features, hog_features)))
    # Return list of feature vectors
    return features





# cars1 = 'vehicles_smallset/cars1/'
# cars2 = 'vehicles_smallset/cars1/'
# cars3 = 'vehicles_smallset/cars3/'
# notcar1 = 'non-vehicles_smallset/notcars1/'
# notcar2 = 'non-vehicles_smallset/notcars2/'
# notcar3 = 'non-vehicles_smallset/notcars3/'              
        
# carsF = [cars1, cars2, cars3]
# cars = []
# for car in carsF:
#     cars.append(glob.glob(car + '*.jpeg'))   
# cars= np.concatenate(cars)

# notcarF = [notcar1, notcar2, notcar3]
# notcars = []
# for notcar in notcarF:
#     notcars.append(glob.glob(notcar + '*.jpeg'))   
# notcars = np.concatenate(notcars)  




cars1 = 'vehicles/GTI_Far/'
cars2 = 'vehicles/GTI_Left/'
cars3 = 'vehicles/GTI_MiddleClose/'
cars4 = 'vehicles/GTI_Right/'
cars5 = 'vehicles/KITTI_extracted/'
notcar1 = 'non-vehicles/GTI/'
notcar2 = 'non-vehicles/Extras/'
             
        
carsF = [cars1, cars2, cars3, cars4, cars5]
cars = []
for car in carsF:
    cars.append(glob.glob(car + '*.png'))   
cars= np.concatenate(cars)

notcarF = [notcar1, notcar2]
# notcarF = [notcar1]
notcars = []
for notcar in notcarF:
    notcars.append(glob.glob(notcar + '*.png'))   
notcars = np.concatenate(notcars) 





color_space = 'RGB' # Can be RGB, HSV, LUV, HLS, YUV, YCrCb
orient = 9  # HOG orientations
pix_per_cell = 8 # HOG pixels per cell
cell_per_block = 2 # HOG cells per block
hog_channel = "0" # Can be 0, 1, 2, or "ALL"
spatial_size = (32, 32) # Spatial binning dimensions
hist_bins = 32    # Number of histogram bins
spatial_feat = True # Spatial features on or off
hist_feat = True # Histogram features on or off
hog_feat = True # HOG features on or off


car_features = extract_features(cars, color_space=color_space, 
                        spatial_size=spatial_size, hist_bins=hist_bins, 
                        orient=orient, pix_per_cell=pix_per_cell, 
                        cell_per_block=cell_per_block, 
                        hog_channel=hog_channel, spatial_feat=spatial_feat, 
                        hist_feat=hist_feat, hog_feat=hog_feat)
notcar_features = extract_features(notcars, color_space=color_space, 
                        spatial_size=spatial_size, hist_bins=hist_bins, 
                        orient=orient, pix_per_cell=pix_per_cell, 
                        cell_per_block=cell_per_block, 
                        hog_channel=hog_channel, spatial_feat=spatial_feat, 
                        hist_feat=hist_feat, hog_feat=hog_feat)

print('length cars', len(car_features))
print('length noncars', len(notcar_features))

X = np.vstack((car_features, notcar_features)).astype(np.float64)                        
# Fit a per-column scaler
X_scaler = StandardScaler().fit(X)
# Apply the scaler to X
scaled_X = X_scaler.transform(X)

# Define the labels vector
y = np.hstack((np.ones(len(car_features)), np.zeros(len(notcar_features))))


# Split up data into randomized training and test sets
rand_state = np.random.randint(0, 100)
X_train, X_test, y_train, y_test = train_test_split(
    scaled_X, y, test_size=0.2, random_state=rand_state)

# X_train = scaled_X
# y_train = y



print('Using:',orient,'orientations',pix_per_cell,
    'pixels per cell and', cell_per_block,'cells per block')
print('Feature vector length:', len(X_train[0]))
print('training data:', len(X_train))
# Use a linear SVC 
svc = LinearSVC()
# Check the training time for the SVC
t=time.time()
svc.fit(X_train, y_train)
t2 = time.time()
print(round(t2-t, 2), 'Seconds to train SVC...')
# # Check the score of the SVC
# print('Test Accuracy of SVC = ', round(svc.score(X_test, y_test), 4))
# # Check the prediction time for a single sample
# t=time.time()

C:\Users\Adam\Anaconda3\lib\site-packages\skimage\feature\_hog.py:119: skimage_deprecation: Default value of `block_norm`==`L1` is deprecated and will be changed to `L2-Hys` in v0.15
  'be changed to `L2-Hys` in v0.15', skimage_deprecation)


length cars 8906
length noncars 9047
Using: 9 orientations 8 pixels per cell and 2 cells per block
Feature vector length: 8460
training data: 14362
8.47 Seconds to train SVC...


In [3]:
# Define a function that takes an image,
# start and stop positions in both x and y, 
# window size (x and y dimensions),  
# and overlap fraction (for both x and y)
def slide_window(img, x_start_stop=[None, None], y_start_stop=[None, None], 
                    xy_window=(64, 64), xy_overlap=(0.5, 0.5)):
    # If x and/or y start/stop positions not defined, set to image size
    if x_start_stop[0] == None:
        x_start_stop[0] = 0
    if x_start_stop[1] == None:
        x_start_stop[1] = img.shape[1]
    if y_start_stop[0] == None:
        y_start_stop[0] = 0
    if y_start_stop[1] == None:
        y_start_stop[1] = img.shape[0]
    # Compute the span of the region to be searched    
    xspan = x_start_stop[1] - x_start_stop[0]
    yspan = y_start_stop[1] - y_start_stop[0]
    # Compute the number of pixels per step in x/y
    nx_pix_per_step = np.int(xy_window[0]*(1 - xy_overlap[0]))
    ny_pix_per_step = np.int(xy_window[1]*(1 - xy_overlap[1]))
    # Compute the number of windows in x/y
    nx_buffer = np.int(xy_window[0]*(xy_overlap[0]))
    ny_buffer = np.int(xy_window[1]*(xy_overlap[1]))
    nx_windows = np.int((xspan-nx_buffer)/nx_pix_per_step) 
    ny_windows = np.int((yspan-ny_buffer)/ny_pix_per_step) 
    # Initialize a list to append window positions to
    window_list = []
    # Loop through finding x and y window positions
    # Note: you could vectorize this step, but in practice
    # you'll be considering windows one by one with your
    # classifier, so looping makes sense
    for ys in range(ny_windows):
        for xs in range(nx_windows):
            # Calculate window position
            startx = xs*nx_pix_per_step + x_start_stop[0]
            endx = startx + xy_window[0]
            starty = ys*ny_pix_per_step + y_start_stop[0]
            endy = starty + xy_window[1]
            
            # Append window position to list
            window_list.append(((startx, starty), (endx, endy)))
    # Return the list of windows
    return window_list

# Define a function to draw bounding boxes
def draw_boxes(img, bboxes, color=(0, 0, 255), thick=6):
    # Make a copy of the image
    imcopy = np.copy(img)
    # Iterate through the bounding boxes
    for bbox in bboxes:
        # Draw a rectangle given bbox coordinates
        cv2.rectangle(imcopy, bbox[0], bbox[1], color, thick)
    # Return the image copy with boxes drawn
    return imcopy



# Define a function to extract features from a single image window
# This function is very similar to extract_features()
# just for a single image rather than list of images
def single_img_features(img, color_space='RGB', spatial_size=(32, 32),
                        hist_bins=32, orient=9, 
                        pix_per_cell=8, cell_per_block=2, hog_channel=0,
                        spatial_feat=True, hist_feat=True, hog_feat=True):    
    #1) Define an empty list to receive features
#     img_features = []

#     features = []
#     feature_image = np.zeros_like(img)
#     r_rgb = img[:,:,0]
#     s_hsv = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)[:,:,1]
#     u_luv = cv2.cvtColor(img, cv2.COLOR_RGB2LUV)[:,:,1]
#     feature_image[:,:,0] = r_rgb
#     feature_image[:,:,1] = s_hsv
#     feature_image[:,:,2] = u_luv
        
    features = []
    feature_image = np.zeros_like(img)
    feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2YCrCb)
    
        #3) Compute spatial features if flag is set
    if spatial_feat == True:
        spatial_features = bin_spatial(feature_image, size=spatial_size)
        #4) Append features to list
#         img_features.append(spatial_features)
    #5) Compute histogram features if flag is set
    if hist_feat == True:
        hist_features = color_hist(feature_image, nbins=hist_bins)
        #6) Append features to list
#         img_features.append(hist_features)
    #7) Compute HOG features if flag is set
    if hog_feat == True:
#         if hog_channel == 'ALL':
        hog_features = []
        for channel in range(feature_image.shape[2]):
            hog_features.extend(get_hog_features(feature_image[:,:,channel], 
                                orient, pix_per_cell, cell_per_block, 
                                vis=False, feature_vec=True))      
#         else:
#             hog_features = get_hog_features(feature_image[:,:,hog_channel], orient, 
#                         pix_per_cell, cell_per_block, vis=False, feature_vec=True)
        #8) Append features to list
#         img_features.append(hog_features)

    features.append(np.concatenate((spatial_features, hist_features, hog_features)))
    #9) Return concatenated array of features
    return features


# Define a function you will pass an image 
# and the list of windows to be searched (output of slide_windows())
def search_windows(img, windows, clf, scaler, color_space='RGB', 
                    spatial_size=(32, 32), hist_bins=32, 
                    hist_range=(0, 256), orient=9, 
                    pix_per_cell=8, cell_per_block=2, 
                    hog_channel=0, spatial_feat=True, 
                    hist_feat=True, hog_feat=True):
    
    #1) Create an empty list to receive positive detection windows
    on_windows = []
    #2) Iterate over all windows in the list
    for window in windows:
        if window:
            #3) Extract the test window from original image
            test_img = cv2.resize(img[window[0][1]:window[1][1], window[0][0]:window[1][0]], (64, 64))      
            #4) Extract features for that window using single_img_features()
            features = single_img_features(test_img, color_space=color_space, 
                                spatial_size=spatial_size, hist_bins=hist_bins, 
                                orient=orient, pix_per_cell=pix_per_cell, 
                                cell_per_block=cell_per_block, 
                                hog_channel=hog_channel, spatial_feat=spatial_feat, 
                                hist_feat=hist_feat, hog_feat=hog_feat)
            #5) Scale extracted features to be fed to classifier
            test_features = scaler.transform(np.array(features).reshape(1, -1))
            #6) Predict using your classifier
            prediction = clf.predict(test_features)
            #7) If positive (prediction == 1) then save the window
            if prediction == 1:
                on_windows.append(window)
    #8) Return windows for positive detections
    return on_windows



def add_heat(heatmap, bbox_list):
    # Iterate through list of bboxes
    for box in bbox_list:
        # Add += 1 for all pixels inside each bbox
        # Assuming each "box" takes the form ((x1, y1), (x2, y2))
        heatmap[box[0][1]:box[1][1], box[0][0]:box[1][0]] += 1

    # Return updated heatmap
    return heatmap# Iterate through list of bboxes
    
def apply_threshold(heatmap, threshold):
    # Zero out pixels below the threshold
    heatmap[heatmap <= threshold] = 0
    # Return thresholded map
    return heatmap

def draw_labeled_bboxes(img, labels):
    # Iterate through all detected cars
    for car_number in range(1, labels[1]+1):
        # Find pixels with each car_number label value
        nonzero = (labels[0] == car_number).nonzero()
        # Identify x and y values of those pixels
        nonzeroy = np.array(nonzero[0])
        nonzerox = np.array(nonzero[1])
        # Define a bounding box based on min/max x and y
        bbox = ((np.min(nonzerox), np.min(nonzeroy)), (np.max(nonzerox), np.max(nonzeroy)))
        # Draw the box on the image
        cv2.rectangle(img, bbox[0], bbox[1], (0,0,255), 6)
    # Return the image
    return img   



# # Read in Video Frames as Images
# images = glob.glob('Data/'+'*.jpg')

# for image in images[300:400]:
#     image = mpimg.imread(image)
#     result = find_vehicles(image)
# #     plt.figure()
# #     plt.imshow(result)

# # labels then heatmap

### Lane Detection Defs

In [4]:
# This method takes an image and the desired gradent orientation
# and comutes a sobel edge detected image in that gradient direction.
def abs_sobel_thresh(img, orient='x', sobel_kernel=3, thresh=(0, 255)):

    # Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)# cv2
    
    # Apply x or y gradient with the OpenCV Sobel() function
    # and take the absolute value

    if orient == 'x':
        abs_sobel = np.absolute(cv2.Sobel(gray, cv2.CV_64F, 1, 0,ksize=sobel_kernel))

    if orient == 'y':
        abs_sobel = np.absolute(cv2.Sobel(gray, cv2.CV_64F, 0, 1,ksize=sobel_kernel))
        
    
    # Rescale back to 8 bit integer
    scaled_sobel = np.uint8(255*abs_sobel/np.max(abs_sobel))

    # Create a copy and apply the threshold
    binary_output = np.zeros_like(scaled_sobel)
    # Here I'm using inclusive (>=, <=) thresholds, but exclusive is ok too
    binary_output[(scaled_sobel >= thresh[0]) & (scaled_sobel <= thresh[1])] = 1  

    return binary_output

# This method calculates the sobel in both x and y 
# and filteres out gradients below a certain magnitude
def mag_thresh(image, sobel_kernel=3, mag_thresh=(0, 255)):
    # Calculate gradient magnitude
    # Apply threshold
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)#cv2
    
    # Take both Sobel x and y gradients
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    # Calculate the gradient magnitude
    gradmag = np.sqrt(sobelx**2 + sobely**2)
    # Rescale to 8 bit
    scale_factor = np.max(gradmag)/255 
    gradmag = (gradmag/scale_factor).astype(np.uint8) 
    # Create a binary image of ones where threshold is met, zeros otherwise
    binary_output = np.zeros_like(gradmag)
    binary_output[(gradmag >= mag_thresh[0]) & (gradmag <= mag_thresh[1])] = 1
    return binary_output

# gradients outside of an expected range of line gradients are thresholded out
def dir_threshold(image, sobel_kernel=3, thresh=(0, np.pi/2)):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # Calculate the x and y gradients
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
    sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel)
    # Take the absolute value of the gradient direction, 
    # apply a threshold, and create a binary image result
    absgraddir = np.arctan2(np.absolute(sobely), np.absolute(sobelx))
    binary_output =  np.zeros_like(absgraddir)
    binary_output[(absgraddir >= thresh[0]) & (absgraddir <= thresh[1])] = 1
    return binary_output

# a hugh light saturation image s chanel is used to pick up lane markings
def hls_threshold(image, s_thresh=(170, 255)):   
    hls = cv2.cvtColor(image, cv2.COLOR_BGR2HLS).astype(np.float)
    s_channel = hls[:,:,2]
    # Threshold saturation channel
    s_binary = np.zeros_like(s_channel)
    s_binary[(s_channel >= s_thresh[0]) & (s_channel <= s_thresh[1])] = 1    
    return s_binary

# the red channel on the RGB spectrum was seen to be effetive in distinquishing the lane markings
def red_threshold(image, r_thresh=(170, 255)):   

    r_channel = image[:,:,2] # using cv2 to read
    # Threshold saturation channel
    r_binary = np.zeros_like(r_channel)
    r_binary[(r_channel >= r_thresh[0]) & (r_channel <= r_thresh[1])] = 1    
    return r_binary

   


def pipeline(image):

    
    # Choose a Sobel kernel size
    ksize = 15 # Choose a larger odd number to smooth gradient measurements

    # Apply each of the thresholding functions
    gradx = abs_sobel_thresh(image, orient='x', sobel_kernel=ksize, thresh=(100, 255))
    grady = abs_sobel_thresh(image, orient='y', sobel_kernel=ksize, thresh=(130, 255))
    mag_binary = mag_thresh(image, sobel_kernel=ksize, mag_thresh=(70, 255))
    dir_binary = dir_threshold(image, sobel_kernel=ksize, thresh=(np.pi/2-1, np.pi/2-.35))
    s_binary = hls_threshold(image, s_thresh=(150, 255))
    red_binary = red_threshold(image, r_thresh=(200, 255))
    combined = np.zeros_like(gradx)
    combined[((gradx == 1) | (grady == 1)) | ((mag_binary == 1) & (dir_binary == 1))|((red_binary == 1)&(s_binary == 1))] = 1

    return combined



# the below function takes in a set of vertices and masks out all pixels outside of this polygon
# The mask region was made slightly larger than the lane itself to insure sharper curves were also included
def maskRegion(image, vertices):
    #defining a blank mask to start with
    mask = np.zeros_like(image) 
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(image.shape) > 2:
        channel_count = image.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255 

    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(image, mask)
    
    return masked_image


# This method returns the perspective transform matrix and its inverse
def get_M_Minv(img):
    img_size = (img.shape[1],img.shape[0])  
    src = np.float32(
    [[578, 460],
    [200, 720],
    [1090, 720],
    [692, 460]])  
    dst = np.float32(
    [[320, 0],
    [320, 720],
    [960, 720],
    [960, 0]])
    # compute perspective transform
    M = cv2.getPerspectiveTransform(src,dst)    
    Minv = cv2.getPerspectiveTransform(dst,src)   
    return M, Minv

# gets perspective transformed image
def warp(img, M):
    img_size = (img.shape[1],img.shape[0])   
    warped = cv2.warpPerspective(img, M, img_size, flags=cv2.INTER_LINEAR)
    return warped

# unwarps the image back to original
def unWarp(img, Minv):
    img_size = (img.shape[1],img.shape[0])    
    warped = cv2.warpPerspective(img, Minv, img_size, flags=cv2.INTER_LINEAR)
    return unWarped


# This methods determines where the base of the lane is using a histogram
def getHist(warped):
    histogram = np.sum(warped[int(warped.shape[0]/2):,:], axis=0)
    return histogram

def extractForFit(warped,LtLine, RtLine): 
    
    histogram = getHist(warped)
    midpoint = np.int(histogram.shape[0]/2)
    nwindows = 9
    # Set height of windows
    window_height = np.int(warped.shape[0]/nwindows)
    nonzero = warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])    
    # Set the width of the windows +/- margin
    margin = 100
    # Set minimum number of pixels found to recenter window
    minpix = 30
    # Create empty lists to receive left and right lane pixel indices
    left_lane_inds = []
    right_lane_inds = []

  
    if not LtLine.detected:
        leftx_base = np.argmax(histogram[:midpoint])
        leftx_current = leftx_base
        # Step through the windows one by one
        for window in range(nwindows):        
            win_y_low = warped.shape[0] - (window+1)*window_height
            win_y_high = warped.shape[0] - window*window_height
            win_xleft_low = leftx_current - margin
            win_xleft_high = leftx_current + margin
            # Identify the nonzero pixels in x and y within the window
            good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high)\
                              & (nonzerox >= win_xleft_low) & (nonzerox < win_xleft_high)).nonzero()[0]

            # Append these indices to the lists
            left_lane_inds.append(good_left_inds)

            # If you found > minpix pixels, recenter next window on their mean position
            if len(good_left_inds) > minpix:
                leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
        # Concatenate the arrays of indices
        left_lane_inds = np.concatenate(left_lane_inds)
    
    else:
#         left_lane_inds = ((nonzerox > (left_fit[0]*(nonzeroy**2) + left_fit[1]*nonzeroy+ left_fit[2] - margin)) & \
#                           (nonzerox < (left_fit[0]*(nonzeroy**2) + left_fit[1]*nonzeroy + left_fit[2] + margin)))
        left_lane_inds = ((nonzerox > (LtLine.best_fit[0]*(nonzeroy**2) + \
                                       LtLine.best_fit[1]*nonzeroy+ LtLine.best_fit[2] - margin)) & \
                  (nonzerox < (LtLine.best_fit[0]*(nonzeroy**2) + \
                               LtLine.best_fit[1]*nonzeroy + LtLine.best_fit[2] + margin)))

        
        
    if not RtLine.detected:
        rightx_base = np.argmax(histogram[midpoint:])+ midpoint
        rightx_current = rightx_base
        # Step through the windows one by one
        for window in range(nwindows):        
            win_y_low = warped.shape[0] - (window+1)*window_height
            win_y_high = warped.shape[0] - window*window_height
            win_xright_low = rightx_current - margin
            win_xright_high = rightx_current + margin
            
            # Identify the nonzero pixels in x and y within the window
            good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high)\
                              & (nonzerox >= win_xright_low) & (nonzerox < win_xright_high)).nonzero()[0]

            # Append these indices to the lists
            right_lane_inds.append(good_right_inds)

            # If you found > minpix pixels, recenter next window on their mean position
            if len(good_right_inds) > minpix:
                rightx_current = np.int(np.mean(nonzerox[good_right_inds]))
        right_lane_inds = np.concatenate(right_lane_inds)
    else:
        right_lane_inds = ((nonzerox > (RtLine.best_fit[0]*(nonzeroy**2) + RtLine.best_fit[1]*nonzeroy + RtLine.best_fit[2] - margin)) \
                           & (nonzerox < (RtLine.best_fit[0]*(nonzeroy**2) + RtLine.best_fit[1]*nonzeroy + RtLine.best_fit[2] + margin)))  



            # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]
    
     

    
    return leftx, lefty, rightx, righty           
       




# Class containing characteristics of each line detection
class Line():
    def __init__(self):
        # was the line detected in the last iteration?
        self.detected = False  
        # x values of the last n fits of the line
        self.recent_xfitted = [] 
        # y values of the last n fits of the line
        self.recent_yfitted = [] 
        #average x values of the fitted line over the last n iterations
        self.bestx = None     
        #polynomial coefficients averaged over the last n iterations
        self.best_fit = None 
        #previos frame best fit
        self.prevbest_fit = None
        #polynomial coefficients for the most recent fit
        self.current_fit = [np.array([False])]  
        #radius of curvature of the line in some units
        self.radius_of_curvature = None         
        #Location where line hits bottom of image
        self.line_base_pos = None 
        #difference in fit coefficients between last and new fits
        self.diffs = np.array([0,0,0], dtype='float') 
        #x values for detected line pixels in last n frames
        self.arrayx = None  
        #y values for detected line pixels in last n frames
        self.array = None
        
        
# This function take drawes a shaded green region to mark the lane
def markLane(image, warped, left_fit,right_fit):
    # y points to mark line
    ploty = np.linspace(0, image.shape[0]-1, num=image.shape[0])
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    color_warp = np.zeros_like(image).astype(np.uint8)

    # Recast the x and y points into usable format for cv2.fillPoly()
    pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])

    pts = np.hstack((pts_left, pts_right))

    # Draw the lane onto the warped blank image
    cv2.fillPoly(color_warp, np.int_([pts]), (0,255, 0))

    # Warp the blank back to original image space using inverse perspective matrix (Minv)
    newwarp = cv2.warpPerspective(color_warp, Minv, (image.shape[1], image.shape[0])) 
    # Combine the result with the original image
    result = cv2.addWeighted(image, 1, newwarp, 0.3, 0)

    return result



def lane_pipelineV(image):
    global loopn
    global curLtLine
    global curRtLine
    global offCenter
    global framesL
    
        # undistor image
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    image = cv2.undistort(image, mtx, dist, None, mtx)
    # detect prospective lines
    combined = pipeline(image)
    # mask out regions outside of expected lane region
    masked = maskRegion(combined,vertices)  
    # perfor merspective transform
    warped = warp(masked, M)
    
    # extract left and right pixel location of perspective lane lines
    leftx, lefty, rightx, righty = extractForFit(warped, curLtLine, curRtLine)
    # get a 2nd order polynomial fit for prospective left and right lines
    
    
    if len(leftx)>500:

        curLtLine.current_fit = np.polyfit(lefty,leftx, 2)
        curLtLine.detected = True
    else:
        curLtLine.detected = False  
        
    if len(rightx)>500:
        curRtLine.current_fit = np.polyfit(righty,rightx, 2)
        curRtLine.detected = True
    else:
        curRtLine.detected = False 

    
    if curLtLine.detected:    
        # keep last n frames of data for poly fit
        if(len(curLtLine.recent_xfitted)<framesL):
            curLtLine.recent_xfitted.append(leftx)
            curLtLine.recent_yfitted.append(lefty)
        else:
            curLtLine.recent_xfitted[loopn%framesL] =leftx 
            curLtLine.recent_yfitted[loopn%framesL] =lefty

        curLtLine.arrayx = np.concatenate(curLtLine.recent_xfitted)
        curLtLine.arrayy = np.concatenate(curLtLine.recent_yfitted)
#         curLtLine.best_fit = np.polyfit(curLtLine.arrayy,\
#                                     curLtLine.arrayx, 2)
        tempFitL = np.polyfit(curLtLine.arrayy,\
                                    curLtLine.arrayx, 2)

        left_base = tempFitL[0]*y_eval**2 + tempFitL[1]*y_eval + tempFitL[2]
        # curvature in meters
        left_fit_m_temp = np.polyfit(curLtLine.arrayy*ym_per_pix, \
                                curLtLine.arrayx*xm_per_pix, 2)
        temp_L_curve= ((1 + (2*left_fit_m_temp[0]*y_eval*ym_per_pix+left_fit_m_temp[1])**2)\
                       **1.5) /np.absolute(2*left_fit_m_temp[0])

        
    if curRtLine.detected:
        # keep last n frames of data for poly fit
        if(len(curRtLine.recent_xfitted)<framesL):
            curRtLine.recent_xfitted.append(rightx)
            curRtLine.recent_yfitted.append(righty)
        else:
            curRtLine.recent_xfitted[loopn%framesL] =rightx 
            curRtLine.recent_yfitted[loopn%framesL] =righty
            
        curRtLine.arrayx = np.concatenate(curRtLine.recent_xfitted)
        curRtLine.arrayy = np.concatenate(curRtLine.recent_yfitted)
        tempFitR = np.polyfit(curRtLine.arrayy,\
                                        curRtLine.arrayx, 2)
        right_base = tempFitR[0]*y_eval**2 + tempFitR[1]*y_eval + tempFitR[2] 
        
        right_fit_m_temp = np.polyfit(curRtLine.arrayy*ym_per_pix, \
                                curRtLine.arrayx*xm_per_pix, 2)
        
        temp_R_curve= ((1 + (2*right_fit_m_temp[0]*y_eval*ym_per_pix+right_fit_m_temp[1])**2)\
                       **1.5) /np.absolute(2*right_fit_m_temp[0])


    # Initial loop
    if loopn is 0 or curRtLine.best_fit is None or curLtLine.best_fit is None:
        if curLtLine.detected:
            curLtLine.best_fit = tempFitL
            curLtLine.line_base_pos = left_base
            curLtLine.radius_of_curvature = temp_L_curve
            left_fit_m = left_fit_m_temp
        if curRtLine.detected:
            curRtLine.best_fit = tempFitR
            right_fit_m = right_fit_m_temp
            curRtLine.radius_of_curvature = temp_R_curve
            curRtLine.line_base_pos = right_base      
    elif curRtLine.detected and curLtLine.detected and curLtLine.best_fit is not None and curRtLine.best_fit is not None:
        laneWidthPx = abs(right_base - left_base)
        # if lane to small or too big only update line that is closest to previous line
        # or if the difference in radius changes by more than 10% of previous only update the closest line
        if laneWidthPx*xm_per_pix < 3 or laneWidthPx*xm_per_pix > 4.4 or\
            abs(temp_R_curve - temp_L_curve) >1.03*abs(curLtLine.radius_of_curvature-curRtLine.radius_of_curvature):
            leftChange = LA.norm(curLtLine.best_fit - curLtLine.current_fit)
            rightChange = LA.norm(curRtLine.best_fit - curRtLine.current_fit)
            if leftChange > rightChange :
                curLtLine.detected = False # false set here so next frame recognizes previos line not good
                curRtLine.best_fit = tempFitR
                right_fit_m = right_fit_m_temp
                curRtLine.radius_of_curvature = temp_R_curve
                curRtLine.line_base_pos = right_base
            else:
                curRtLine.detected = False
                curLtLine.best_fit = tempFitL
                left_fit_m = left_fit_m_temp
                curLtLine.radius_of_curvature = temp_L_curve
                curLtLine.line_base_pos = left_base
        else:
            curRtLine.best_fit = tempFitR
            right_fit_m = right_fit_m_temp        
            curLtLine.best_fit = tempFitL
            left_fit_m = left_fit_m_temp        
            curRtLine.radius_of_curvature = temp_R_curve
            curLtLine.radius_of_curvature = temp_L_curve
            curRtLine.line_base_pos = right_base
            curLtLine.line_base_pos = left_base
    
#     loopn = loopn+1
    if curLtLine.best_fit is not None and curRtLine.best_fit is not None:
        laneWidthPx = curRtLine.line_base_pos - curLtLine.line_base_pos
        lane_center = laneWidthPx/2+curLtLine.line_base_pos
        offCenter = (lane_center - imgmid)*xm_per_pix        
        sign = np.sign(offCenter)

        if sign>0:
            side = "Vehicle Right of Center by "+str(offCenter)+"m"
        else:
            side = "Vehicle Left of Center by "+str(offCenter)+"m"
        
        if(curLtLine.best_fit[0] - curLtLine.line_base_pos)>0:
            direct = "Right"
        else:
            direct = "Left"
        radiusWords = "Radius of curvature is "+str(curLtLine.radius_of_curvature) + "m to the " + str(direct)
        result = markLane(image, warped, curLtLine.best_fit, curRtLine.best_fit)#should use best fit
#         cv2.putText(result, side, (50,100), cv2.FONT_HERSHEY_SIMPLEX, 1,(255,255,255),4)
#         cv2.putText(result, radiusWords, (50,200), cv2.FONT_HERSHEY_SIMPLEX, 1,(255,255,255),4)  
        
        result = cv2.cvtColor(result, cv2.COLOR_BGR2RGB)
        return result
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    return image





In [ ]:
import os

import numpy as np
from numpy import linalg as LA
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

# Camera calibration matrices
mtx= np.array([[  1.15396093e03,   0.00000000e00,   6.69705359e02],\
                [  0.00000000e00,   1.14802495e03,   3.85656232e02],\
                [  0.00000000e00,   0.00000000e00,   1.00000000e00]])
dist = np.array([[ -2.41017968e-01 , -5.30720497e-02,\
                  -1.15810318e-03,  -1.28318543e-04,   2.67124302e-02]])

offCenter = 0 # in meters the distance the car is from the center of the lane
curLtLine = Line()
curRtLine = Line()
framesL=4
 
# loopn = 0 
test = 'test_images/bbox-example-image.jpg'
image = cv2.imread(test)
# get perspective transform matrices
M, Minv = get_M_Minv(image)
img_size = (image.shape[1],image.shape[0])
# bottom left, to top left, to top right, to bottom right
vertices = np.array([[((img_size[0] / 6 - 130), img_size[1]),\
                     ((img_size[0] / 2) - 85, img_size[1] / 2 + 100),\
                     ((img_size[0] / 2 + 85), img_size[1] / 2 + 100),\
                     ((img_size[0] * 5 / 6) + 130, img_size[1])]],dtype=np.int32)
# of pixels per meter to get radius in meters
ym_per_pix = 30/720
xm_per_pix = 3.7/700

# evaluate curvature 
y_eval =image.shape[0]
imgmid = image.shape[1]/2



box_sizes = [(256,256),(200,200),(164,164),(128,128),(96,96)]
y_start_stop = [[380, 700],[360,600],[380, 550],[380, 550],[400,500]]
overlaps = [(.5,.5),(.5,.7),(.7,.7),(.9,.4),(.8,.8)]
# changed search region from 350 to 400 and reduced threshold to 1


image = mpimg.imread('test_images/frame0.jpg')
windows = []
for b_size, y_lim,overlap in zip(box_sizes,y_start_stop, overlaps):
    windows.extend(slide_window(image, x_start_stop=[None, None], y_start_stop=y_lim, 
                        xy_window=b_size, xy_overlap=overlap))

frames = 2
box_group  = []
loopn = 0
def find_vehicles_lanes(img):
    global box_group
    global loopn
    global windows  
    # find lane_markings
    lane_img =lane_pipelineV(img)
    

    box_list = []
    hot_windows = []
#     image = img
    image = img.astype(np.float32)/255
    hot_windows = search_windows(image, windows, svc, X_scaler, color_space=color_space, 
                            spatial_size=spatial_size, hist_bins=hist_bins, 
                            orient=orient, pix_per_cell=pix_per_cell, 
                            cell_per_block=cell_per_block, 
                            hog_channel=hog_channel, spatial_feat=spatial_feat, 
                            hist_feat=hist_feat, hog_feat=hog_feat)                       
    heat = np.zeros_like(image[:,:,0]).astype(np.float)
#     box_list = hot_windows

    if hot_windows:
        if(len(box_group)<frames):
            box_group.append(hot_windows)
        else:
            box_group[loopn%frames] = hot_windows 
    for group in box_group:
        box_list.extend(group)

    # Add heat to each box in box list
    heat = add_heat(heat,box_list)
    heatmap2 = np.clip(heat, 0, 255)
    # Apply threshold to help remove false positives
    heat = apply_threshold(heat,3)

    # Visualize the heatmap when displaying    
    heatmap = np.clip(heat, 0, 255)

    # Find final boxes from heatmap using label function
    labels = label(heatmap)
    draw_img = draw_labeled_bboxes(np.copy(lane_img), labels)
    loopn = loopn +1
    
#     fig = plt.figure()
#     plt.subplot(121)
#     plt.imshow(draw_img)
#     plt.title('Car Positions')
#     plt.subplot(122)
#     plt.imshow(heatmap2, cmap='hot')
#     plt.title('Heat Map')
#     fig.tight_layout()
    
   
    return draw_img

from moviepy.editor import VideoFileClip
from IPython.display import HTML

outputVid8 = 'Vehicles_Detected.mp4'
clip = VideoFileClip("original_highway.mp4")
lane_clip = clip.fl_image(find_vehicles_lanes) #NOTE: this function expects color images!!
lane_clip.write_videofile(outputVid8, audio=False)

[MoviePy] >>>> Building video Vehicles_Detected.mp4
[MoviePy] Writing video Vehicles_Detected.mp4


  0%|                                                                                         | 0/1261 [00:00<?, ?it/s]

In [12]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(outputVid8))